# Pre-Filtering
The first approach we could take is to pre-filter our vectors. This consists of taking our metadata index and applying our set of conditions. From here, we return a list of all records in our vector index that satisfies our filter conditions.

Now when we search, we’ve restricted our scope — so our vector similarity search will be faster, and we’ll only return relevant results!

The problem is that our pre-filter disrupts how ANN engines work (ANN requires the full index), leaving us with two options:

Build an ANN index for every possible filter. Perform a brute-force kNN search across remaining vectors.

Option (1) is simply not practical, there are far too many possible filter outputs in a typical index.

That leaves us with option (2), creating the additional overhead of brute-force checking every single vector embedding remaining after the metadata filter.

# Post Filtering
We can’t rely on a brute-force search every time we apply a filter, so what if we applied the filter after our vector search?

Working through it, we have our vector index and a query — we perform a similarity search as usual. We return k of the top nearest matches. We’ll set k == 10.

We’ve now got our top 10 best matches, but there’s plenty of vectors in here that would not satisfy our filter conditions.

We go ahead and apply our post-search filter to remove those irrelevant results. That filter has removed six of our results, leaving us with just four results. We wanted ten results, and we’ve returned four. That’s not so great.


Unfortunately, it gets worse… What if we didn’t return any results in the top k that satisfy our filter conditions? That leaves us with no results at all.

We can eliminate the risk of returning too few (or zero) results by increasing k to a high number. But now we have slower search times thanks to our excessively high k searches, and so we’re back to the slow search times of pre-filtering.

---------------
Slow search with pre-filtering, or unreliable results with post-filtering. Neither of these approaches sounds particularly attractive, so what can we do?


# Single Stage Filtering
Pinecone’s single-stage filtering produces the accurate results of pre-filtering at even faster speeds than post-filtering.

It works by merging the vector and metadata indexes into a single index — resulting in a single-stage filter as opposed to the two-stage filter-and-search method of pre- and post-filtering.

This gives us pre-filtering accuracy benefits without being restricted to small datasets. It can even increase search speeds whether you have a dataset of 1 million or 100 billion.

In [ ]:
# RUN THIS CELL ONLY ONCE TO DOWNLOAD THE DATASETS
from datasets import load_dataset

# Load English SQuAD (v1.1 or v2)
squad_en = load_dataset("rajpurkar/squad") 

# Load Italian SQuAD (SQuAD-it)
squad_it = load_dataset("crux82/squad_it")

C:\Users\thareddy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Save the datasets to local folder for future use
squad_en.save_to_disk("./data/squad_en")

squad_it.save_to_disk("./data/squad_it")

Saving the dataset (1/1 shards): 100%|██████████| 7609/7609 [00:00<00:00, 312002.85 examples/s]


In [1]:
from datasets import load_from_disk

# Load the datasets back from the local folder
squad_en = load_from_disk("./data/squad_en")
squad_it = load_from_disk("./data/squad_it")

C:\Users\thareddy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
squad_it['train'][1]

{'id': '56cdca7862d2951400fa6828',
 'context': "Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.",
 'question': 'Quante persone sono state uccise come risultato?',
 'answers': {'text': ['69.197'], 'answer_start': [232]}}

In [24]:
data = []
for item in squad_en['validation']:
    data.append({
        'id': item['id'],
        'context': item['context'],
        'metadata':{
            "lang" : "en",
            'title': item['title']
        }
    })
for item in squad_it['test']:
    data.append({
        'id': item['id'],
        'context': item['context'],
        'metadata':{
            "lang" : "it",
            'title': '' if 'title' not in item else item['title']
        }
    })

    # we are only taking validation data becuase, converting the train data nto embedding is taking lot of time



In [25]:
len(data)

18179

In [26]:
data[0].keys()

dict_keys(['id', 'context', 'metadata'])

For Pinecone, we need three items: id, vector, and metadata. We have two of those, but we’re missing our vector field.

In [27]:
from sentence_transformers import SentenceTransformer
import os
from tqdm import tqdm

# Setup
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
embedder = SentenceTransformer('stsb-xlm-r-multilingual')

# Configuration
batch_size = 32  #
contexts = [item['context'] for item in data]

vectors = embedder.encode(
    contexts, 
    batch_size=batch_size, 
    show_progress_bar=True, 
    convert_to_numpy=True
)

# Mapping vectors back to the original data list
for i, item in enumerate(data):
    item['vector'] = vectors[i]

Batches: 100%|██████████| 569/569 [17:26<00:00,  1.84s/it]


In [28]:
data[0].keys()

dict_keys(['id', 'context', 'metadata', 'vector'])

In [29]:
data[0]

{'id': '56be4db0acb8001400a502ec',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'metadata': {'lang': 'en', 'title': 'Super_Bowl_50'},
 'vector': array([-5.80232680e-01,  1.32507354e-01,  3.04604352e-01, -1.50068123e-02,
        -2.39832684e-01, -7.2177

In [30]:
with open('squad_embedded_data.pkl', 'wb') as f:
    import pickle
    pickle.dump(data, f)

In [31]:
with open('squad_embedded_data.pkl', 'rb') as f:
    import pickle
    data = pickle.load(f)

In [32]:
data[0].keys()

dict_keys(['id', 'context', 'metadata', 'vector'])